In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
def draw_registration_result(source, target, transformation=None):
    if transformation is None:
        transformation = np.asarray([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def execute_global_registration(
        source_down,
        target_down,
        source_fpfh,
        target_fpfh,
        distance_threshold
):
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source=source_down,
        target=target_down,
        source_feature=source_fpfh,
        target_feature=target_fpfh,
        mutual_filter=False,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        # estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(200000, 0.99999)
    )
    return result


def execute_global_registration_corr(source_down, target_down, corres, distance_threshold=None):
    if distance_threshold is None:
        distance_threshold = radius(source_down) * 1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
        source_down,
        target_down,
        corres,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3,
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(200000, 0.99999)
    )
    return result


def preprocess_point_cloud(pcd, voxel_size=0):
    # pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down = pcd

    radius_ = radius(pcd)
    radius_normal = radius_ * 3
    radius_feature = radius_ * 20
    max_nn = 200
    # radius_feature = voxel_size * 5
    # radius_normal = voxel_size * 2

    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn))
    pcd_down.estimate_normals()
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))
    return pcd_down, pcd_fpfh


def show_range_coords(ply):
    ply_np = np.asarray(ply.points)
    print()
    for i in range(3):
        print(
            ply_np[:, i].min(),
            ply_np[:, i].max(),
            "range:",
            ply_np[:, i].max() - ply_np[:, i].min()
        )


def radius(ply):
    distances = ply.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    print("radius:", radius)
    return radius


def get_scale_factor(source, target, corres):
    source_np = np.asarray(source.points)
    target_np = np.asarray(target.points)

    t_d_0 = np.linalg.norm(target_np[corres[0][1]] - target_np[corres[1][1]])
    t_d_1 = np.linalg.norm(target_np[corres[1][1]] - target_np[corres[2][1]])

    s_d_0 = np.linalg.norm(source_np[corres[0][0]] - source_np[corres[1][0]])
    s_d_1 = np.linalg.norm(source_np[corres[1][0]] - source_np[corres[2][0]])
    # print(t_d_0, s_d_0, t_d_0/s_d_0)
    # print(t_d_1, s_d_1, t_d_1/s_d_1)
    scale_ratio_0 = float(t_d_0/s_d_0)
    scale_ratio_1 = float(t_d_1/s_d_1)

    return sum([scale_ratio_0, scale_ratio_1])/2


def slice(ply, threshold, preview=False):
    """
    :param threshold: percentage from z_min
    """
    assert 0 < threshold < 1
    ply_np = np.asarray(ply.points)
    z_min = ply_np[:, 2].min()
    z_max = ply_np[:, 2].max()
    z_threshold = (z_max - z_min) * threshold

    ply_sliced = ply.select_by_index(np.where(ply_np[:, 2] < z_min + z_threshold)[0])

    print(ply)
    print(ply_sliced)
    print(1, z_min + z_threshold)

    if preview:
        o3d.visualization.draw_geometries([ply_sliced])
    return ply_sliced

In [18]:
# ply_source = "D:/sfm_dense/street_1_0_real_time/target.ply"
# ply_source = "D:/sfm_dense/street_2_0/dense/0/color_pd30_8bit_sample.ply"

# ply_target = "D:/sfm_dense/street_1_0_query/dense/0/fused.ply"
# ply_target = "D:/sfm_dense/street_1_0_query/query.sparse.ply"
# ply_target ="E:/University/Thesis/color_pd30_8bit_small_filtered.ply"

# street_4_0
# ply_source = "D:/sfm_dense/street_4_0_1/dense/0/fused_scaled_uniformed.ply"
# ply_target = "D:/sfm_dense/color_pd30_8bit_street_4_0_sliced_uniformed.ply"

# street_4_3_2
# success
# ply_source = "D:/sfm_dense/street_4_3_2/dense/0/street_4_3_2_fused_ele_uniformed_aligned.ply"
# ply_target = "D:\sfm_dense\street_4_3_2\street_4_3_2_color_pd30_8bit_small_ele_uniformed_aligned.ply"

# street_4_3_3
# success
ply_source = "D:/sfm_dense/street_4_3_3/dense/0/fused_uniformed_aligned.ply"
# ply_target = "D:\sfm_dense\street_4_3_3\color_pd30_8bit_small_street_4_3_3.ply"
ply_target = "D:\sfm_dense\street_4_3_3\color_pd30_8bit_small_street_4_3_3_ele_uniformed_sliced.ply"


source = o3d.io.read_point_cloud(ply_source)
target = o3d.io.read_point_cloud(ply_target)

# Preprocessing source
print("source")
scale_ = 10
scale_ = 11.49  # 4_3_3
show_range_coords(source)
# source = slice(source, 0.15)  # NO for corres
source.scale(scale_, center=source.get_center())
# source = source.voxel_down_sample(voxel_size=radius(source)*1.5)  # NO for corres


# Preprocessing target
print("target")
show_range_coords(target)
# target = slice(target, 0.5)


draw_registration_result(source, target)

source_down, source_fpfh = preprocess_point_cloud(source)
target_down, target_fpfh = preprocess_point_cloud(target)

source

-5.18475866317749 5.746166229248047 range: 10.930924892425537
-6.182560920715332 3.956705331802368 range: 10.1392662525177
-0.5147021412849426 1.6505919694900513 range: 2.165294110774994
target

-81.13799711303918 85.67450288696082 range: 166.8125
-76.10136783277241 68.89863216722759 range: 145.0
-11.236785264176856 -5.636782975358497 range: 5.6000022888183585
radius: 0.6639217821248121
radius: 2.891610618263282


In [13]:
source_down = source_down.translate([100, 100, 0], relative=False)

result_fast = execute_global_registration(
    source_down,
    target_down,
    source_fpfh,
    target_fpfh,
    radius(source)*1.5
)
print("Global", result_fast)
draw_registration_result(source_down, target_down, result_fast.transformation)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, radius(source)*1.5, result_fast.transformation)
print("Global", evaluation)
print("Global Transformation",)
print(result_fast.transformation)
print()

radius: 2.104506917465689
Global RegistrationResult with fitness=5.274361e-01, inlier_rmse=1.631254e+00, and correspondence_set size of 1115
Access transformation to get result.
radius: 2.104506917465689
Global RegistrationResult with fitness=5.274361e-01, inlier_rmse=1.631254e+00, and correspondence_set size of 1115
Access transformation to get result.
Global Transformation
[[ 3.94911840e-01 -9.18650236e-01  1.12420199e-02  8.99363575e+01]
 [-9.18685945e-01 -3.94764224e-01  1.33169915e-02  1.45858695e+02]
 [-7.79571014e-03 -1.55869233e-02 -9.99848126e-01 -5.91778817e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [19]:
target_indexes = [
    5572,
    4510,
    2124,
    2624
]
source_indexes = [
    1811,
    1984,
    9395,
    566
]
corres = o3d.utility.Vector2iVector(list(zip(source_indexes, target_indexes)))
result = execute_global_registration_corr(
    source_down,
    target_down,
    corres,
    distance_threshold=None
)
print("Global Corres", result)
draw_registration_result(source_down, target_down, result.transformation)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, radius(source)*1.5, result.transformation)
print("Global Corres", evaluation)
print("Global Corres Transformation",)
print(result.transformation)
print()

radius: 0.6639217821248121
Global Corres RegistrationResult with fitness=2.470229e-01, inlier_rmse=5.856566e-01, and correspondence_set size of 7675
Access transformation to get result.
radius: 0.6639217821248121
Global Corres RegistrationResult with fitness=2.470229e-01, inlier_rmse=5.856566e-01, and correspondence_set size of 7675
Access transformation to get result.
Global Corres Transformation
[[-8.28591373e-01 -5.59158000e-01  2.79046348e-02  7.52762634e+00]
 [ 5.59333007e-01 -8.28941076e-01 -1.81082177e-03 -2.28787170e+01]
 [ 2.41438335e-02  1.41075520e-02  9.99608950e-01 -4.43274391e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [24]:
print(source_fpfh)
print(source_fpfh.data.shape)

Feature class with dimension = 33 and num = 31070
Access its data via data member.
(33, 31070)


In [89]:
threshold = 1
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source_down,
    target_down,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=6.626889e-01, inlier_rmse=5.223891e-01, and correspondence_set size of 833
Access transformation to get result.
Transformation is:
[[ 5.44446215e-01 -8.38774711e-01  5.94160171e-03  2.47639086e+01]
 [ 8.38505340e-01  5.44057675e-01 -3.01668849e-02 -1.49167377e+02]
 [ 2.20706462e-02  2.14063111e-02  9.99527216e-01 -1.26211217e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [90]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source,
    target,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=6.603023e-01, inlier_rmse=5.096639e-01, and correspondence_set size of 830
Access transformation to get result.
Transformation is:
[[ 6.60372333e-01 -7.50935691e-01  1.99245699e-03  3.08888396e+00]
 [ 7.50631980e-01  6.60024821e-01 -3.03127997e-02 -1.54736644e+02]
 [ 2.14478921e-02  2.15133362e-02  9.99538476e-01 -1.25494599e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

